In [1]:
import os.path
print(os.path.exists(r'dataset/single_prediction/cat_or_dog_1.jpg'))

True


In [ ]:
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from keras.models import Sequential
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


In [ ]:
classifier=Sequential()
#convolution neural network

#building the CNN
classifier.add(
   Convolution2D(32, #no of filter
                (3,3),#kernel size
                input_shape=(64,64,3),
                activation='relu',
                data_format='channels_last'))


In [ ]:
#step2 max pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))  
classifier.add(
        Convolution2D(32,(3,3),activation='relu',data_format='channels_last'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=1,activation='sigmoid'))
classifier.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])


In [ ]:
#training data generate
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
training_set=train_datagen.flow_from_directory('dataset/Train',
                                              target_size=(64,64),
                                              batch_size=32,
                                              class_mode='binary')
test_set=test_datagen.flow_from_directory('dataset/Test',
                                              target_size=(64,64),
                                              batch_size=32,
                                              class_mode='binary')
sess=tf.Session(config=tf.ConfigProto(log_device_placement=True))
with tf.device('/GPU:0'):
    classifier.fit_generator(training_set,
                            steps_per_epoch=8000,
                            epochs=1,
                            validation_data=test_set,
                            validation_steps=2000)


In [ ]:
#testing 
test_image=image.load_img(path=r'dataset/single_prediction/cat_or_dog_1.jpg',
                         target_size=(64,64))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=classifier.predict(test_image)
indices=training_set.class_indices
if result[0][0]==1:
    prediction='dog'
else:
    prediction='cat'